# Install Lib

In [ ]:
# Install required libraries
!pip install -q transformers datasets scikit-learn accelerate pandas openpyxl huggingface_hub

# Import Dependancies

In [ ]:
# Handle imports
import pandas as pd
from google.colab import drive
from datasets import Dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import EarlyStoppingCallback
from huggingface_hub import notebook_login

# Load Data

In [ ]:
# Explore data
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/arabic-author-style-datasets/heldout_10percent_per_author.xlsx")
df_10_each = df.groupby('author').head(10).reset_index(drop=True)

df_10_each

In [ ]:
texts = df_10_each['text_in_author_style'].tolist()
labels = df_10_each['author'].tolist()

# Encode authors
le = LabelEncoder()
labels = le.fit_transform(labels)


✅ Goal
Train a lightweight classifier on each BERT layer to see how well it predicts the author — this tells you which layers carry the most relevant signal for authorship.



# Tokenize

In [ ]:
from transformers import BertTokenizer

model_id = "aubmindlab/bert-base-arabertv2"
model_id = "Tami3/arabic-author-classifier"

tokenizer = BertTokenizer.from_pretrained(model_id)
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)


# Extract Hidden States

In [ ]:
from transformers import BertModel
import torch

model = BertModel.from_pretrained(model_id, output_hidden_states=True)
model.eval()

with torch.no_grad():
    outputs = model(**encodings)
    hidden_states = outputs.hidden_states  # tuple of 13 tensors [embeddings + 12 layers]


# Collect [CLS] Embeddings Layer-Wise

In [ ]:
cls_per_layer = []

# Loop through each layer
for i in range(1, 13):  # skip embeddings (index 0)
    cls_embeddings = hidden_states[i][:, 0, :]  # CLS token
    cls_per_layer.append(cls_embeddings.cpu().numpy())


# Train a Simple Classifier for Each Layer

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

accuracies = []
f1_macros = []

X_train, X_test, y_train, y_test = train_test_split(np.zeros((len(texts), 768)), labels, test_size=0.2)

for i, cls_embeddings in enumerate(cls_per_layer):
    X_train, X_test, y_train, y_test = train_test_split(cls_embeddings, labels, test_size=0.2, random_state=42)
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average='macro')
    print(f"Layer {i+1} Accuracy: {acc:.4f} | F1 Macro: {f1_macro:.4f}")
    f1_macros.append(f1_macro)
    accuracies.append(acc)


# Plot Results

🔍 What This Tells You

*   Layers with higher accuracy are likely carrying more author-specific signals.

*   Earlier layers capture syntax and general grammar.

* Later layers typically encode more task-specific features — like authorship.




In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, 13), accuracies, marker='o', label='Accuracy')
plt.plot(range(1, 13), f1_macros, marker='s', label='F1 Macro')
plt.xlabel("BERT Layer")
plt.ylabel("Score")
plt.title("Layer-wise Author Classification Accuracy and F1 Macro")
plt.legend()
plt.grid(True)
plt.show()


🔍 1. Attention Analysis
> We’ll inspect self-attention scores to see where BERT focuses across the input — does it attend more to stylistic phrases or syntactic structure that might signal authorship?

In [ ]:
from transformers import BertModel, BertTokenizer
import matplotlib.pyplot as plt
import torch

# Enable attention outputs
model = BertModel.from_pretrained(model_id, output_attentions=True)
model.eval()

text = df_10_each['text_in_author_style'].iloc[0]
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)

with torch.no_grad():
    outputs = model(**inputs)

# attention: tuple of (layers, batch, heads, tokens, tokens)
attentions = outputs.attentions  # len 12

# Visualize attention from a specific head in a specific layer
layer = 11  # Last layer
head = 0

attn_matrix = attentions[layer][0, head].cpu().numpy()  # shape [seq_len, seq_len]
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

plt.rcParams['font.family'] = 'DejaVu Sans'

plt.figure(figsize=(10, 8))
plt.imshow(attn_matrix, cmap="viridis")
plt.xticks(range(len(tokens)), tokens, rotation=90)
plt.yticks(range(len(tokens)), tokens)
plt.title(f"Layer {layer+1} Head {head+1} Self-Attention")
plt.colorbar()
plt.tight_layout()
plt.show()


🔁 2. CKA (Centered Kernel Alignment)
> CKA helps compare how similar the representation spaces are across layers or authors.

🔹 You can also compute CKA across different authors' embeddings to see where their representations differ.





In [ ]:
import numpy as np

def linear_cka(X, Y):
    X = X - X.mean(0)
    Y = Y - Y.mean(0)
    dot_product_similarity = np.linalg.norm(X.T @ Y, 'fro') ** 2
    normalization_x = np.linalg.norm(X.T @ X, 'fro')
    normalization_y = np.linalg.norm(Y.T @ Y, 'fro')
    return dot_product_similarity / (normalization_x * normalization_y)

num_layers = len(cls_per_layer)
cka_matrix = np.zeros((num_layers, num_layers))
for i in range(num_layers):
    for j in range(num_layers):
        cka_matrix[i, j] = linear_cka(np.array(cls_per_layer[i]), np.array(cls_per_layer[j]))


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.imshow(cka_matrix, cmap='plasma', vmin=0, vmax=1)
plt.title("CKA Similarity Between BERT Layers")
plt.colorbar()
plt.xlabel("Layer")
plt.ylabel("Layer")
plt.xticks(range(num_layers), range(1, num_layers + 1))
plt.yticks(range(num_layers), range(1, num_layers + 1))
plt.tight_layout()
plt.show()


🌈 3. t-SNE Visualization
> Visualize how well BERT separates different authors in embedding space using 2D projections.



In [ ]:
plt.figure(figsize=(10, 6))
scatter = sns.scatterplot(
    x=X_embedded[:, 0],
    y=X_embedded[:, 1],
    hue=le.inverse_transform(labels),
    palette='tab10',
    alpha=0.7
)
plt.title("t-SNE of BERT CLS Representations (Layer 10)")

# Move legend outside to the right
plt.legend(
    title='Author',           # Or whatever your classes represent
    bbox_to_anchor=(1.05, 1), # (x, y) relative to axes (1.05 means a bit right of the plot)
    loc='upper left',         # anchor to the upper left of the bounding box
    borderaxespad=0.
)
plt.tight_layout() # Adjust plot to fit legend
plt.show()
